# Chapter 16

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS
from scipy import stats

In [2]:
# Exercise 1
smoke = pd.read_stata("./stata/SMOKE.DTA")
X = sm.add_constant(smoke[["cigs", "educ", "age", "agesq"]])
model = sm.OLS(smoke.lincome, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lincome   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     39.61
Date:                Sun, 22 May 2022   Prob (F-statistic):           2.68e-30
Time:                        20:14:55   Log-Likelihood:                -798.50
No. Observations:                 807   AIC:                             1607.
Df Residuals:                     802   BIC:                             1630.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.7954      0.170     45.741      0.000       7.461       8.130
cigs           0.0017      0.002      1.010      0.313      -0.002       0.005
educ           0.0604      0.008      7.642      0.000       0.045       0.076
age            0.0577      0.008      7.548      0.000       0.043       0.073
agesq         -0.0006   8.34e-05     -7.563      0.000      -0.001      -0.000
==============================================================================
Omnibus:                      264.025   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              960.743
Skew:                          -1.531   Prob(JB):                    2.38e-209
Kurtosis:                       7.381   Cond. No.                     1.88e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [3]:
X = sm.add_constant(smoke[["educ", "age", "agesq", "lcigpric", "restaurn"]])
model = sm.OLS(smoke.cigs, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cigs   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     8.610
Date:                Sun, 22 May 2022   Prob (F-statistic):           5.86e-08
Time:                        20:14:55   Log-Likelihood:                -3237.0
No. Observations:                 807   AIC:                             6486.
Df Residuals:                     801   BIC:                             6514.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5801     23.696      0.067      0.947     -44.933      48.093
educ          -0.4501      0.162     -2.785      0.005      -0.767      -0.133
age            0.8225      0.154      5.330      0.000       0.520       1.125
agesq         -0.0096      0.002     -5.711      0.000      -0.013      -0.006
lcigpric      -0.3513      5.766     -0.061      0.951     -11.669      10.966
restaurn      -2.7364      1.110     -2.466      0.014      -4.915      -0.558
==============================================================================
Omnibus:                      226.788   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              500.148
Skew:                           1.543   Prob(JB):                    2.48e-109
Kurtosis:                       5.313   Cond. No.                     1.31e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
X = sm.add_constant(smoke[["educ", "age", "agesq"]])
iv = smoke[["lcigpric", "restaurn"]]
IV2SLS(smoke.lincome, X, smoke.cigs, iv).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                lincome   R-squared:                     -0.5169
Estimator:                    IV-2SLS   Adj. R-squared:                -0.5245
No. Observations:                 807   F-statistic:                    89.802
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:14:55   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          7.7809     0.2292     33.955     0.0000      7.3318      8.2300
educ           0.0397     0.0162     2.4444     0.0145      0.0079      0.0715
age            0.0938     0.0238     3.9454     0.0001      0.0472      0.1404
agesq         -0.0011     0.0003    -3.8424     0.0001     -0.0016     -0.0005
cigs          -0.0421     0.0261    -1.6117     0.1070     -0.0934      0.0091
==============================================================================

Endogenous: cigs
Instruments: lcigpric, restaurn
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae48cb20

C1.i We are regressing log(income) and so the coefficient ($\beta_1$) is the income elasticity with respect to cigrettes. That is, the percent change in income per cigarette smoked per day.

C1.ii The price of a pack of ciagrettes would likely be negatively related to consumption, though they are addictive so this is not assured. We should expect either a negative or zero coefficient. Smoking restrictions are, by definition, intended to reduce smoking and so again we would expect negative or zero.

C1.iii We require at least one exogenous variable and so either the (log) price of a pack or the restaurant restriction should have a non-zero coefficient.

C1.iv The estimate on cigs is positive but small and insignificant.

C1.v The smoking restriction variable is significant in the reduced form (at the 5% level) but not the log price of a pack.

C1.vi The coefficient for cigs is now negative but is still not significant, even at the 10% level. The effect is somewhat large (at least 4 times more than the estimate from OLS in absolute terms. Also a 4% drop in income is large in general).

C1.vii Restaurant restrictions are likely to vary state by state as would incomes. This does not seem entirely in line with the assumption of exogeneity (that is, if restrictions on smoking are more or less likely in states with higher incomes, this is a problem).

In [5]:
# Exercise 2
mroz = pd.read_stata("./stata/MROZ.DTA")
mroz["lhours"] = np.log(mroz.hours)
mroz = mroz[["lhours", "lwage", "educ", "age", "kidslt6", "nwifeinc", "exper",
             "expersq", "motheduc", "fatheduc"]].dropna()
X = sm.add_constant(mroz[["educ", "age", "kidslt6", "nwifeinc"]])
iv = mroz[["exper", "expersq"]]
IV2SLS(mroz.lhours, X, mroz.lwage, iv).fit(cov_type="unadjusted")

/home/systemchalk/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                 lhours   R-squared:                     -1.7762
Estimator:                    IV-2SLS   Adj. R-squared:                -1.8091
No. Observations:                 428   F-statistic:                    24.395
Date:                Sun, May 22 2022   P-value (F-stat)                0.0002
Time:                        20:14:55   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          8.3702     0.6842     12.234     0.0000      7.0293      9.7112
educ          -0.2355     0.0704    -3.3459     0.0008     -0.3734     -0.0975
age           -0.0135     0.0112    -1.2111     0.2258     -0.0354      0.0084
kidslt6       -0.4654     0.2178    -2.1368     0.0326     -0.8924     -0.0385
nwifeinc      -0.0139     0.0079    -1.7653     0.0775     -0.0293      0.0015
lwage          1.9943     0.5603     3.5592     0.0004      0.8961      3.0926
==============================================================================

Endogenous: lwage
Instruments: exper, expersq
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae452b20

In [6]:
X = sm.add_constant(mroz[["age", "kidslt6", "nwifeinc"]])
iv = mroz[["exper", "expersq", "motheduc", "fatheduc"]]
results = IV2SLS(mroz.lhours, X, mroz[["lwage", "educ"]], iv).fit(
    cov_type="unadjusted")
results

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                 lhours   R-squared:                     -1.4820
Estimator:                    IV-2SLS   Adj. R-squared:                -1.5114
No. Observations:                 428   F-statistic:                    26.282
Date:                Sun, May 22 2022   P-value (F-stat)                0.0001
Time:                        20:14:56   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          7.2608     1.0122     7.1731     0.0000      5.2768      9.2447
age           -0.0116     0.0105    -1.1037     0.2697     -0.0322      0.0090
kidslt6       -0.5432     0.2098    -2.5885     0.0096     -0.9545     -0.1319
nwifeinc      -0.0189     0.0087    -2.1677     0.0302     -0.0360     -0.0018
lwage          1.8109     0.4943     3.6638     0.0002      0.8422      2.7797
educ          -0.1286     0.0868    -1.4813     0.1385     -0.2988      0.0416
==============================================================================

Endogenous: lwage, educ
Instruments: exper, expersq, motheduc, fatheduc
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae3fabe0

In [7]:
results.wooldridge_overid

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 0.6487
P-value: 0.7230
Distributed: chi2(2)
WaldTestStatistic, id: 0x7f3aae410d00

In [8]:
overid_fit = sm.OLS(results.resids, mroz[["age", "kidslt6", "nwifeinc", 
                                          "exper", "expersq", "motheduc",
                                          "fatheduc"]]).fit()
overid_fit.summary()
print("Manual test p-value:", 
      stats.chi2.sf(overid_fit.rsquared * overid_fit.nobs, 2))

Manual test p-value: 0.7999839798648982


C2.i Labour supply elasticity raises now to about 1.99 (up from 1.26).

C2.ii Results above, estimate for lwage falls slightly to 1.81 but is still higher than before.

C2.iii The test fails to reject the null meaning we have not found evidence that any of our IVs are not exogenous. The IVs pass the test.

In [9]:
# Exercise 3
openness = pd.read_stata("./stata/OPENNESS.DTA")
X = sm.add_constant(openness[["open"]])
sm.OLS(openness.inf, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    inf   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     5.311
Date:                Sun, 22 May 2022   Prob (F-statistic):             0.0230
Time:                        20:14:56   Log-Likelihood:                -520.90
No. Observations:                 114   AIC:                             1046.
Df Residuals:                     112   BIC:                             1051.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.2342      4.102      6.152      0.000      17.106      33.362
open          -0.2150      0.093     -2.304      0.023      -0.400      -0.030
==============================================================================
Omnibus:                      162.333   Durbin-Watson:                   2.106
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6623.109
Skew:                           5.343   Prob(JB):                         0.00
Kurtosis:                      38.779   Cond. No.                         81.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
iv = openness[["lland"]]
IV2SLS(openness.inf, np.ones(openness.shape[0]), openness.open, iv).fit(
    cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                    inf   R-squared:                      0.0316
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0230
No. Observations:                 114   F-statistic:                    5.7259
Date:                Sun, May 22 2022   P-value (F-stat)                0.0167
Time:                        20:14:56   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog           29.607     5.6084     5.2790     0.0000      18.614      40.599
open          -0.3329     0.1391    -2.3929     0.0167     -0.6055     -0.0602
==============================================================================

Endogenous: open
Instruments: lland
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae3b9fa0

In [11]:
sm.OLS(openness.open, sm.add_constant(openness["land"])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   open   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     11.76
Date:                Sun, 22 May 2022   Prob (F-statistic):           0.000848
Time:                        20:14:56   Log-Likelihood:                -516.69
No. Observations:                 114   AIC:                             1037.
Df Residuals:                     112   BIC:                             1043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         40.4499      2.342     17.271      0.000      35.809      45.090
land       -1.128e-05   3.29e-06     -3.429      0.001   -1.78e-05   -4.76e-06
==============================================================================
Omnibus:                       72.146   Durbin-Watson:                   2.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              361.210
Skew:                           2.180   Prob(JB):                     3.67e-79
Kurtosis:                      10.552   Cond. No.                     7.84e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.84e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
sm.OLS(openness.open, sm.add_constant(openness["lland"])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   open   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     90.90
Date:                Sun, 22 May 2022   Prob (F-statistic):           3.93e-16
Time:                        20:14:56   Log-Likelihood:                -488.51
No. Observations:                 114   AIC:                             981.0
Df Residuals:                     112   BIC:                             986.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        121.8385      9.044     13.472      0.000     103.919     139.758
lland         -7.6182      0.799     -9.534      0.000      -9.201      -6.035
==============================================================================
Omnibus:                       55.789   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              193.082
Skew:                           1.747   Prob(JB):                     1.18e-42
Kurtosis:                       8.333   Cond. No.                         62.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
sm.OLS(openness.open, sm.add_constant(openness[["land", "lland"]])
      ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   open   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     46.77
Date:                Sun, 22 May 2022   Prob (F-statistic):           1.85e-15
Time:                        20:14:56   Log-Likelihood:                -487.54
No. Observations:                 114   AIC:                             981.1
Df Residuals:                     111   BIC:                             989.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        129.2173     10.471     12.341      0.000     108.468     149.966
land        4.334e-06   3.14e-06      1.382      0.170   -1.88e-06    1.05e-05
lland         -8.3978      0.975     -8.609      0.000     -10.331      -6.465
==============================================================================
Omnibus:                       54.521   Durbin-Watson:                   2.172
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              178.858
Skew:                           1.729   Prob(JB):                     1.45e-39
Kurtosis:                       8.069   Cond. No.                     4.53e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.53e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
X = sm.add_constant(openness[["oil", "lpcinc"]])
iv = openness[["lland"]]
IV2SLS(openness.inf, X, openness.open, iv).fit(
    cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                    inf   R-squared:                      0.0349
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0086
No. Observations:                 114   F-statistic:                    6.2377
Date:                Sun, May 22 2022   P-value (F-stat)                0.1006
Time:                        20:14:57   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          24.009     15.752     1.5242     0.1275     -6.8638      54.882
oil           -6.5557     9.6279    -0.6809     0.4959     -25.426      12.315
lpcinc         0.8033     2.0804     0.3861     0.6994     -3.2743      4.8809
open          -0.3370     0.1419    -2.3743     0.0176     -0.6151     -0.0588
==============================================================================

Endogenous: open
Instruments: lland
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae382fa0

C3.i Results above. The result is not very different from the result reported in the book.

C3.ii The log land variable explains more variation than the untransformed variable (as indicated by $R^2$). When running a regression with both variables the transformed variable is significant.

C3.iii The estimate suggests a large negative relationship between oil and inflation (greater than 6%), but the result is not significant.

In [15]:
# Exercise 4
consump = pd.read_stata("./stata/consump.dta")
consump = consump[["gy", "r3", "gc", "gc_1", "gy_1", "r3_1"]].dropna()
X = consump[["gy", "r3"]]
iv = consump[["gc_1", "gy_1", "r3_1"]]
iv_results = IV2SLS(consump.gc, np.ones(consump.shape[0]), X, iv).fit(
    cov_type="unadjusted")
iv_results

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                     gc   R-squared:                      0.6779
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6578
No. Observations:                  35   F-statistic:                    20.970
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:14:57   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog           0.0081     0.0031     2.6074     0.0091      0.0020      0.0141
gy             0.5862     0.1287     4.5555     0.0000      0.3340      0.8384
r3            -0.0003     0.0007    -0.3688     0.7123     -0.0017      0.0012
==============================================================================

Endogenous: gy, r3
Instruments: gc_1, gy_1, r3_1
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae3b9760

In [16]:
iv_results.wooldridge_overid

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 2.0386
P-value: 0.1534
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f3aae401100

In [17]:
overid_fit = sm.OLS(iv_results.resids, sm.add_constant(iv)).fit()
overid_fit.summary()
print("Manual test p-value:", 
      stats.chi2.sf(overid_fit.rsquared * overid_fit.nobs, 1))

Manual test p-value: 0.14291384678903202


In [18]:
consump = pd.read_stata("./stata/consump.dta")
consump = consump[["gy", "r3", "gc", "gc_2", "gy_2", "r3_2"]].dropna()
X = consump[["gy", "r3"]]
iv = consump[["gc_2", "gy_2", "r3_2"]]
iv_results = IV2SLS(consump.gc, np.ones(consump.shape[0]), X, iv).fit(
    cov_type="unadjusted")
iv_results

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                     gc   R-squared:                     -0.1161
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1881
No. Observations:                  34   F-statistic:                    1.0528
Date:                Sun, May 22 2022   P-value (F-stat)                0.5907
Time:                        20:14:57   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog          -0.0054     0.0261    -0.2080     0.8352     -0.0566      0.0458
gy             1.2042     1.2144     0.9916     0.3214     -1.1759      3.5844
r3            -0.0004     0.0019    -0.2278     0.8198     -0.0041      0.0032
==============================================================================

Endogenous: gy, r3
Instruments: gc_2, gy_2, r3_2
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae4596a0

In [19]:
sm.OLS(consump.gy, sm.add_constant(iv)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     gy   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.085
Method:                 Least Squares   F-statistic:                    0.1390
Date:                Sun, 22 May 2022   Prob (F-statistic):              0.936
Time:                        20:14:57   Log-Likelihood:                 88.045
No. Observations:                  34   AIC:                            -168.1
Df Residuals:                      30   BIC:                            -162.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0208      0.007      3.170      0.003       0.007       0.034
gc_2          -0.0702      0.469     -0.150      0.882      -1.029       0.888
gy_2           0.0937      0.330      0.284      0.778      -0.580       0.768
r3_2           0.0007      0.002      0.445      0.659      -0.003       0.004
==============================================================================
Omnibus:                        0.143   Durbin-Watson:                   1.582
Prob(Omnibus):                  0.931   Jarque-Bera (JB):                0.094
Skew:                           0.102   Prob(JB):                        0.954
Kurtosis:                       2.843   Cond. No.                         425.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

C4.i The test fails to reject the null at even the 10% level and so it passes. We do not have evidence to suggest our IVs are exogenous.

C4.ii The coefficient for gy is larger but no longer significant. This is a fairly substantial change since the PIH was rejected in 16.36.

C4.iii None of the individual coefficients are statistically significant. This is less important than the fact that the F-Statistic is small and so the second lags are not sufficiently correlated with gy. This is important because we expect our IVs to be correlated with the endogenous variable!

Note: Omitting Exercise 5. Subsequent reports (website goes back to 2010, others available through Federal Reserve) use a different base for GDP so in addition to adding > 25 years of data (100+ additional data points by hand), real values need to be updated. I note that the R package "wooldridge" which claims to be updated for the 7th edition only includes the consump data set from this edition and so it may have been dropped from later editions. I'll try to get back to this at a later date but I would rather work on new problems.

In [20]:
# Exercise 6
cement = pd.read_stata("./stata/cement.dta")
X = sm.add_constant(cement[["gcem", "gprcpet", "feb", "mar", "apr", "may",
                            "jun", "jul", "aug", "sep", "oct", "nov", "dec"]])
model = sm.OLS(cement.gprc, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   gprc   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     13.72
Date:                Sun, 22 May 2022   Prob (F-statistic):           1.35e-23
Time:                        20:14:58   Log-Likelihood:                 924.74
No. Observations:                 298   AIC:                            -1821.
Df Residuals:                     284   BIC:                            -1770.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0144      0.003      4.516      0.000       0.008       0.021
gcem          -0.0443      0.009     -4.866      0.000      -0.062      -0.026
gprcpet        0.0628      0.015      4.111      0.000       0.033       0.093
feb           -0.0034      0.005     -0.714      0.476      -0.013       0.006
mar            0.0009      0.005      0.158      0.875      -0.010       0.012
apr            0.0055      0.005      1.045      0.297      -0.005       0.016
may           -0.0087      0.004     -1.964      0.050      -0.017    1.88e-05
jun           -0.0109      0.005     -2.411      0.017      -0.020      -0.002
jul           -0.0111      0.004     -3.036      0.003      -0.018      -0.004
aug           -0.0098      0.004     -2.300      0.022      -0.018      -0.001
sep           -0.0165      0.004     -4.527      0.000      -0.024      -0.009
oct           -0.0147      0.004     -3.690      0.000      -0.022      -0.007
nov           -0.0265      0.003     -8.295      0.000      -0.033      -0.020
dec           -0.0302      0.003     -9.359      0.000      -0.037      -0.024
==============================================================================
Omnibus:                      160.517   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1624.081
Skew:                           1.973   Prob(JB):                         0.00
Kurtosis:                      13.735   Cond. No.                         25.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
X = sm.add_constant(cement[["gdefs", "gprcpet", "feb", "mar", "apr", "may",
                            "jun", "jul", "aug", "sep", "oct", "nov", "dec"]])
model = sm.OLS(cement.gcem, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   gcem   R-squared:                       0.858
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     135.7
Date:                Sun, 22 May 2022   Prob (F-statistic):          2.38e-115
Time:                        20:14:58   Log-Likelihood:                 375.50
No. Observations:                 306   AIC:                            -723.0
Df Residuals:                     292   BIC:                            -670.9
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2482      0.015    -16.873      0.000      -0.277      -0.219
gdefs         -1.0541      3.580     -0.294      0.769      -8.099       5.991
gprcpet        0.0670      0.100      0.672      0.502      -0.129       0.263
feb            0.3972      0.021     19.233      0.000       0.357       0.438
mar            0.4870      0.021     23.586      0.000       0.446       0.528
apr            0.4631      0.020     22.635      0.000       0.423       0.503
may            0.3446      0.021     16.804      0.000       0.304       0.385
jun            0.3568      0.020     17.494      0.000       0.317       0.397
jul            0.2051      0.020     10.021      0.000       0.165       0.245
aug            0.3133      0.020     15.296      0.000       0.273       0.354
sep            0.2037      0.021      9.894      0.000       0.163       0.244
oct            0.2636      0.021     12.759      0.000       0.223       0.304
nov            0.0100      0.021      0.487      0.627      -0.031       0.051
dec           -0.0606      0.021     -2.947      0.003      -0.101      -0.020
==============================================================================
Omnibus:                       38.708   Durbin-Watson:                   2.472
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              238.282
Skew:                          -0.191   Prob(JB):                     1.81e-52
Kurtosis:                       7.306   Cond. No.                         895.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
X = sm.add_constant(cement[["gres", "gnon", "gprcpet", "feb", "mar", "apr",
                            "may", "jun", "jul", "aug", "sep", "oct", "nov",
                            "dec"]])
model = sm.OLS(cement.gcem, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   gcem   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     143.3
Date:                Sun, 22 May 2022   Prob (F-statistic):          3.24e-122
Time:                        20:14:58   Log-Likelihood:                 396.69
No. Observations:                 309   AIC:                            -763.4
Df Residuals:                     294   BIC:                            -707.4
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2437      0.014    -17.484      0.000      -0.271      -0.216
gres           0.1361      0.138      0.984      0.326      -0.136       0.409
gnon           1.1455      0.210      5.466      0.000       0.733       1.558
gprcpet        0.0369      0.095      0.391      0.696      -0.149       0.223
feb            0.3976      0.019     20.504      0.000       0.359       0.436
mar            0.4709      0.020     23.716      0.000       0.432       0.510
apr            0.4576      0.019     23.551      0.000       0.419       0.496
may            0.3402      0.019     17.556      0.000       0.302       0.378
jun            0.3555      0.019     18.409      0.000       0.318       0.394
jul            0.1977      0.019     10.175      0.000       0.159       0.236
aug            0.3084      0.019     15.883      0.000       0.270       0.347
sep            0.1878      0.019      9.646      0.000       0.149       0.226
oct            0.2705      0.019     13.948      0.000       0.232       0.309
nov            0.0031      0.020      0.158      0.875      -0.035       0.042
dec           -0.0713      0.020     -3.644      0.000      -0.110      -0.033
==============================================================================
Omnibus:                       27.126   Durbin-Watson:                   2.601
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              111.440
Skew:                          -0.122   Prob(JB):                     6.33e-25
Kurtosis:                       5.932   Cond. No.                         56.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
cement = cement[["gprc", "gres", "gnon", "gprcpet", "feb", "mar", "apr",
                 "may", "jun", "jul", "aug", "sep", "oct", "nov",
                 "dec", "gcem"]].dropna()
X = sm.add_constant(cement[["gprcpet", "feb", "mar", "apr", "may", "jun",
                            "jul", "aug", "sep", "oct", "nov", "dec"]])
iv = cement[["gres", "gnon"]]
IV2SLS(cement.gprc, X, cement.gcem, iv).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                   gprc   R-squared:                      0.3560
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3265
No. Observations:                 298   F-statistic:                    154.94
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:14:58   Distribution:                 chi2(13)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0228     0.0071     3.2147     0.0013      0.0089      0.0367
gprcpet        0.0605     0.0154     3.9404     0.0001      0.0304      0.0906
feb           -0.0168     0.0112    -1.5009     0.1334     -0.0388      0.0051
mar           -0.0156     0.0136    -1.1472     0.2513     -0.0423      0.0111
apr           -0.0101     0.0129    -0.7810     0.4348     -0.0353      0.0152
may           -0.0201     0.0097    -2.0714     0.0383     -0.0391     -0.0011
jun           -0.0230     0.0102    -2.2570     0.0240     -0.0430     -0.0030
jul           -0.0180     0.0063    -2.8345     0.0046     -0.0304     -0.0055
aug           -0.0204     0.0091    -2.2486     0.0245     -0.0382     -0.0026
sep           -0.0234     0.0063    -3.6928     0.0002     -0.0358     -0.0110
oct           -0.0236     0.0078    -3.0149     0.0026     -0.0389     -0.0082
nov           -0.0269     0.0032    -8.3908     0.0000     -0.0332     -0.0206
dec           -0.0283     0.0035    -8.0221     0.0000     -0.0352     -0.0214
gcem          -0.0106     0.0271    -0.3899     0.6966     -0.0636      0.0425
==============================================================================

Endogenous: gcem
Instruments: gres, gnon
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae30cfa0

C6.i We should expect $\alpha_1$ to be positive since an increase in price should entice producers to make more concrete. $\beta_1$ should also be positive since increases in the cost of an input to production would be presumably passed on. Unexpectedly $\alpha_1 < 0$ (and is statistically significant) when estimated by OLS which would mean we have a downward sloping supply curve (!?).

C6.ii Running a regression shows that we have no evidence that the coefficient for growth in defence spending is different from zero and so it is not a suitable IV.

C6.iii gnon is correlated, but we have no evidence that gres is correlated.

C6.iv The coefficient for gcem is small and insiginifcant (and still negative), but gprcpet is positive and significant. This would suggest a horizontal supply curve.

In [24]:
# Exercise 7
crime = pd.read_stata("./stata/CRIME4.DTA")
X = sm.add_constant(crime[["d83", "d84", "d85", "d86", "d87", "clprbarr",
                           "clprbcon", "clprbpri", "clavgsen", "cltaxpc"]])
model = sm.OLS(crime.clpolpc, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                clpolpc   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     10.29
Date:                Sun, 22 May 2022   Prob (F-statistic):           6.10e-16
Time:                        20:14:59   Log-Likelihood:                -11.154
No. Observations:                 540   AIC:                             44.31
Df Residuals:                     529   BIC:                             91.51
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0390      0.028     -1.400      0.162      -0.094       0.016
d83            0.0972      0.038      2.526      0.012       0.022       0.173
d84            0.0529      0.038      1.393      0.164      -0.022       0.128
d85            0.0350      0.038      0.917      0.360      -0.040       0.110
d86            0.0369      0.039      0.946      0.344      -0.040       0.114
d87            0.0909      0.039      2.312      0.021       0.014       0.168
clprbarr       0.1149      0.048      2.380      0.018       0.020       0.210
clprbcon       0.2480      0.027      9.020      0.000       0.194       0.302
clprbpri       0.0803      0.042      1.918      0.056      -0.002       0.163
clavgsen      -0.0682      0.036     -1.914      0.056      -0.138       0.002
cltaxpc        0.0052      0.065      0.080      0.936      -0.122       0.133
==============================================================================
Omnibus:                      366.421   Durbin-Watson:                   2.130
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49589.486
Skew:                          -2.002   Prob(JB):                         0.00
Kurtosis:                      49.776   Cond. No.                         7.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

C7.i If there is an anticipated increase in crime, it would be reasonable to assume more police will be hired to counteract it. This would explain a positive relationship between crime and police.

C7.ii The question would be if taxes collected are related to the crime rate. On its own there's no intuitive reason to think so, though there is a possibility that its relationship to the health of the economy (more taxes when the economy is healthy) may cause some relationship to emerge.

C7.iii The reduced form is estimated above. cltaxpc is not a good candidate for an IV since it does not appear to be correlated with cpolpc.

C7.iv The ideal would be if the grands were assigned randomly. This would be unrelated with the error but would obviously be correlated with clpolpc and so be a suitable IV.

In [25]:
# Exercise 8
fish = pd.read_stata("./stata/FISH.DTA")
X = sm.add_constant(fish[["wave2", "wave3", "mon", "tues", "wed", "thurs"]])
model = sm.OLS(fish.lavgprc, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lavgprc   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     6.555
Date:                Sun, 22 May 2022   Prob (F-statistic):           9.06e-06
Time:                        20:14:59   Log-Likelihood:                -31.772
No. Observations:                  97   AIC:                             77.54
Df Residuals:                      90   BIC:                             95.57
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0228      0.144     -7.096      0.000      -1.309      -0.736
wave2          0.0945      0.021      4.439      0.000       0.052       0.137
wave3          0.0526      0.020      2.652      0.009       0.013       0.092
mon           -0.0121      0.114     -0.106      0.916      -0.238       0.214
tues          -0.0090      0.112     -0.080      0.936      -0.231       0.213
wed            0.0505      0.112      0.453      0.652      -0.171       0.272
thurs          0.1242      0.111      1.121      0.265      -0.096       0.344
==============================================================================
Omnibus:                        7.581   Durbin-Watson:                   0.751
Prob(Omnibus):                  0.023   Jarque-Bera (JB):                7.106
Skew:                          -0.588   Prob(JB):                       0.0286
Kurtosis:                       3.612   Cond. No.                         41.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
model.f_test("wave2 = wave3 = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=19.099814525773915, p=1.21901300890693e-07, df_denom=90, df_num=2>

In [27]:
X = sm.add_constant(fish[["mon", "tues", "wed", "thurs"]])
iv = fish[["wave2", "wave3"]]
IV2SLS(fish.ltotqty, X, fish.lavgprc, iv).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                ltotqty   R-squared:                      0.1933
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1490
No. Observations:                  97   F-statistic:                    23.508
Date:                Sun, May 22 2022   P-value (F-stat)                0.0003
Time:                        20:14:59   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          8.1641     0.1760     46.387     0.0000      7.8191      8.5090
mon           -0.3074     0.2220    -1.3848     0.1661     -0.7426      0.1277
tues          -0.6847     0.2189    -3.1282     0.0018     -1.1138     -0.2557
wed           -0.5206     0.2165    -2.4042     0.0162     -0.9450     -0.0962
thurs          0.0948     0.2181     0.4344     0.6640     -0.3328      0.5223
lavgprc       -0.8158     0.3171    -2.5724     0.0101     -1.4374     -0.1942
==============================================================================

Endogenous: lavgprc
Instruments: wave2, wave3
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae31de50

In [28]:
u_1 = IV2SLS(fish.ltotqty, X, fish.lavgprc, iv).fit(
    cov_type="unadjusted").resids.shift(1)
fish["u_1"] = u_1
fish = fish[["ltotqty", "lavgprc", "mon", "tues", "wed", "thurs", "u_1",
             "wave2", "wave3"]].dropna()

iv = fish[["wave2", "wave3"]]
X = sm.add_constant(fish[["mon", "tues", "wed", "thurs", "u_1"]])
IV2SLS(fish.ltotqty, X, fish.lavgprc, iv).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                ltotqty   R-squared:                      0.2519
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2015
No. Observations:                  96   F-statistic:                    33.448
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:14:59   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          8.1480     0.1693     48.128     0.0000      7.8162      8.4799
mon           -0.2930     0.2183    -1.3421     0.1796     -0.7208      0.1349
tues          -0.6953     0.2118    -3.2826     0.0010     -1.1105     -0.2802
wed           -0.5204     0.2095    -2.4842     0.0130     -0.9310     -0.1098
thurs          0.1003     0.2110     0.4753     0.6346     -0.3132      0.5137
u_1            0.2942     0.0994     2.9586     0.0031      0.0993      0.4890
lavgprc       -0.8741     0.2996    -2.9178     0.0035     -1.4613     -0.2870
==============================================================================

Endogenous: lavgprc
Instruments: wave2, wave3
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae319940

In [29]:
model.f_test("mon=tues=wed=thurs=0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=0.5308457339410264, p=0.7133561345709756, df_denom=90, df_num=4>

C8.i We need at least one exogenous variable that is in the supply equation.

C8.ii We need wave 2 and wave 3 to be excluded from the demand equation and one of them needs to appear in the supply function.

C8.iii wave2 and wave3 are jointly significant. p-value above (but very, very small)

C8.iv The confidence interval is -1.47 to -0.17 with a point estimate of -0.82. The point estimate seems reasonable, but the confidence interval is quite wide (though still significant).

C8.v The lagged residual is significant and so this is evidence of serial correlation (value of 0.29 so not strong correlation but not zero).

C8.vi We would need to assume the dummies are not present in the supply equation but are in the demand equation.

V8.vii The day of the week dummies are not jointly significant and so we are not going to be able to estimate the supply equation without any more information.

In [30]:
# Exercise 9
airfare = pd.read_stata("./stata/airfare.dta")
airfare = airfare[airfare.year == 1997]
X = sm.add_constant(airfare[["lfare", "ldist", "ldistsq"]])
model = sm.OLS(airfare.lpassen, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lpassen   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     22.96
Date:                Sun, 22 May 2022   Prob (F-statistic):           1.96e-14
Time:                        20:15:00   Log-Likelihood:                -1406.0
No. Observations:                1149   AIC:                             2820.
Df Residuals:                    1145   BIC:                             2840.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.2296      2.100      6.299      0.000       9.109      17.350
lfare         -0.3912      0.067     -5.817      0.000      -0.523      -0.259
ldist         -1.5705      0.629     -2.499      0.013      -2.804      -0.337
ldistsq        0.1158      0.048      2.426      0.015       0.022       0.209
==============================================================================
Omnibus:                       66.992   Durbin-Watson:                   1.660
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.790
Skew:                           0.589   Prob(JB):                     7.78e-18
Kurtosis:                       3.509   Cond. No.                     4.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
X = sm.add_constant(airfare[["ldist", "ldistsq", "concen"]])
model = sm.OLS(airfare.lfare, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lfare   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     262.6
Date:                Sun, 22 May 2022   Prob (F-statistic):          1.15e-129
Time:                        20:15:00   Log-Likelihood:                -441.65
No. Observations:                1149   AIC:                             891.3
Df Residuals:                    1145   BIC:                             911.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1901      0.890      6.956      0.000       4.444       7.936
ldist         -0.9361      0.272     -3.443      0.001      -1.469      -0.403
ldistsq        0.1081      0.021      5.240      0.000       0.068       0.149
concen         0.3950      0.063      6.299      0.000       0.272       0.518
==============================================================================
Omnibus:                       25.322   Durbin-Watson:                   1.424
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.904
Skew:                           0.193   Prob(JB):                     0.000129
Kurtosis:                       2.526   Cond. No.                     4.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
X = sm.add_constant(airfare[["ldist", "ldistsq"]])
IV2SLS(airfare.lpassen, X, airfare.lfare, airfare.concen).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                lpassen   R-squared:                     -0.0549
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0577
No. Observations:                1149   F-statistic:                    40.594
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:15:00   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          18.014     3.2169     5.5998     0.0000      11.709      24.319
ldist         -2.1757     0.7261    -2.9965     0.0027     -3.5987     -0.7526
ldistsq        0.1870     0.0612     3.0569     0.0022      0.0671      0.3069
lfare         -1.1740     0.3879    -3.0262     0.0025     -1.9344     -0.4136
==============================================================================

Endogenous: lfare
Instruments: concen
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae247be0

C9.i $\alpha_1$ should be negative since an increase in prices should be associated with fewer passengers.

C9.ii OLS results above. Estimate is -0.39 and significant at any meaningful level.

C9.iii It must be the case that demand only depends on air fare. Essentially a passenger should not care about their airline's market share.

C9.iv Results above, concentration is positive and significant.

C9.v The IV estimate is even larger at -1.17 and still significant at the 1% level.

C9.vi With the inclusion of the squared term the relationship between log passengers and distance is an upward sloping u shape. The minimum is around 5.8.

In [33]:
# Exercise 10
airfare = pd.read_stata("./stata/airfare.dta")
airfare.set_index(["id", "year"], inplace=True)
X = sm.add_constant(airfare[["lfare"]])
PanelOLS(airfare.lpassen, X, entity_effects=True, time_effects=True).fit()

Dep. Variable:,lpassen,R-squared:,0.4278
Estimator:,PanelOLS,R-squared (Between):,-0.0701
No. Observations:,4596,R-squared (Within):,0.2621
Date:,"Sun, May 22 2022",R-squared (Overall):,-0.0583
Time:,20:15:00,Log-likelihood,3082.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2574.4
Entities:,1149,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(1,3443)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,2574.4


In [34]:
X = sm.add_constant(airfare[["concen"]])
PanelOLS(airfare.lfare, X, entity_effects=True, time_effects=True).fit()

Dep. Variable:,lfare,R-squared:,0.0095
Estimator:,PanelOLS,R-squared (Between):,-0.0417
No. Observations:,4596,R-squared (Within):,0.0019
Date:,"Sun, May 22 2022",R-squared (Overall):,-0.0394
Time:,20:15:01,Log-likelihood,4435.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,32.965
Entities:,1149,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(1,3443)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,32.965


In [35]:
for var in ["lfare", "lpassen", "concen", "y98", "y99", "y00"]:
    airfare[f"{var}_dm"] = airfare[var] - airfare[var].groupby("id").mean()

X = airfare[["y98_dm", "y99_dm", "y00_dm"]]

IV2SLS(airfare.lpassen_dm, X, airfare.lfare_dm, airfare.concen_dm).fit(
    cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:             lpassen_dm   R-squared:                      0.2265
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2258
No. Observations:                4596   F-statistic:                    239.51
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:15:01   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
y98_dm         0.0257     0.0085     3.0373     0.0024      0.0091      0.0423
y99_dm         0.0724     0.0104     6.9525     0.0000      0.0520      0.0928
y00_dm         0.1128     0.0238     4.7330     0.0000      0.0661      0.1595
lfare_dm      -0.3016     0.2401    -1.2561     0.2091     -0.7722      0.1690
==============================================================================

Endogenous: lfare_dm
Instruments: concen_dm
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7f3aae2779a0

C10.i The estimated elasticity is about -1.16 and is statistically significant.

C10.ii There is a positive and significant relationship between concen and lfare. If concen is uncorrelated with the errors then it is a suitable IV.

C10.iii The estimated elasticity drops to -0.30 and is not statistically significant.

In [36]:
# Exercise 11
expendshares = pd.read_stata("./stata/expendshares.dta")
expendshares.describe()

,sfood,sfuel,sclothes,salcohol,stransport,sother,totexpend,income,age,kids,ltotexpend,lincome,agesq
count,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000,1519.000000
mean,0.356459,0.091013,0.107232,0.060596,0.132350,0.252349,98.696511,136.247531,35.778802,1.608953,4.512714,4.840407,1340.500329
std,0.105125,0.051902,0.094808,0.063325,0.105322,0.103296,43.192884,61.061164,7.772866,0.488146,0.388861,0.376993,595.860454
min,0.057100,0.000000,0.000000,0.000000,0.000000,0.036100,30.000000,20.000000,19.000000,1.000000,3.401197,2.995732,361.000000
25%,0.281700,0.055300,0.032150,0.012550,0.056950,0.178550,70.000000,100.000000,30.000000,1.000000,4.248495,4.605170,900.000000
50%,0.354000,0.080000,0.085700,0.042300,0.115300,0.239700,90.000000,120.000000,35.000000,2.000000,4.499810,4.787492,1225.000000
75%,0.425750,0.113650,0.158250,0.089950,0.179400,0.310000,120.000000,160.000000,40.000000,2.000000,4.787492,5.075174,1600.000000
max,0.789000,0.480300,0.760200,0.428100,0.763800,0.806500,390.000000,1110.000000,60.000000,2.000000,5.966147,7.012115,3600.000000


In [37]:
X = sm.add_constant(expendshares[["ltotexpend", "age", "kids"]])
model = sm.OLS(expendshares.sfood, X, missing="drop").fit(cov_type="HC3")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sfood   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     195.0
Date:                Sun, 22 May 2022   Prob (F-statistic):          6.55e-107
Time:                        20:15:01   Log-Likelihood:                 1522.9
No. Observations:                1519   AIC:                            -3038.
Df Residuals:                    1515   BIC:                            -3017.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8959      0.029     30.627      0.000       0.839       0.953
ltotexpend    -0.1459      0.006    -23.304      0.000      -0.158      -0.134
age            0.0018      0.000      6.098      0.000       0.001       0.002
kids           0.0343      0.005      7.322      0.000       0.025       0.043
==============================================================================
Omnibus:                       16.303   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.926
Skew:                           0.128   Prob(JB):                     1.73e-05
Kurtosis:                       3.530   Cond. No.                         453.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [38]:
X = sm.add_constant(expendshares[["lincome", "age", "kids"]])
model = sm.OLS(expendshares.ltotexpend, X, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ltotexpend   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     174.0
Date:                Sun, 22 May 2022   Prob (F-statistic):           6.23e-97
Time:                        20:15:02   Log-Likelihood:                -495.25
No. Observations:                1519   AIC:                             998.5
Df Residuals:                    1515   BIC:                             1020.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9223      0.114     16.862      0.000       1.699       2.146
lincome        0.4778      0.024     20.281      0.000       0.432       0.524
age            0.0049      0.001      4.262      0.000       0.003       0.007
kids           0.0644      0.018      3.642      0.000       0.030       0.099
==============================================================================
Omnibus:                       38.268   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.775
Skew:                           0.329   Prob(JB):                     1.89e-10
Kurtosis:                       3.523   Cond. No.                         498.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
X = sm.add_constant(expendshares[["age", "kids"]])
IV2SLS(expendshares.sfood, X, expendshares.ltotexpend, expendshares.lincome
      ).fit(cov_type="robust")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                  sfood   R-squared:                      0.2837
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2822
No. Observations:                1519   F-statistic:                    188.48
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:15:02   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.9523     0.0540     17.629     0.0000      0.8464      1.0582
age            0.0019     0.0003     6.1526     0.0000      0.0013      0.0025
kids           0.0354     0.0047     7.5307     0.0000      0.0262      0.0447
ltotexpend    -0.1600     0.0129    -12.411     0.0000     -0.1853     -0.1347
==============================================================================

Endogenous: ltotexpend
Instruments: lincome
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7f3aadefea60

In [40]:
X = sm.add_constant(expendshares[["lincome", "age", "kids"]])
model = sm.OLS(expendshares.ltotexpend, X).fit()
expendshares["v"] = model.resid

X = sm.add_constant(expendshares[["v", "ltotexpend", "age", "kids"]])
model = sm.OLS(expendshares.sfood, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sfood   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     152.3
Date:                Sun, 22 May 2022   Prob (F-statistic):          1.46e-109
Time:                        20:15:02   Log-Likelihood:                 1523.7
No. Observations:                1519   AIC:                            -3037.
Df Residuals:                    1514   BIC:                            -3011.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9523      0.054     17.661      0.000       0.847       1.058
v              0.0179      0.015      1.216      0.224      -0.011       0.047
ltotexpend    -0.1600      0.013    -12.253      0.000      -0.186      -0.134
age            0.0019      0.000      5.980      0.000       0.001       0.003
kids           0.0354      0.005      7.380      0.000       0.026       0.045
==============================================================================
Omnibus:                       15.678   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.471
Skew:                           0.134   Prob(JB):                     3.59e-05
Kurtosis:                       3.502   Cond. No.                         921.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
model = sm.OLS(expendshares.sfood, X).fit(cov_type="HC3")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sfood   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     153.9
Date:                Sun, 22 May 2022   Prob (F-statistic):          1.59e-110
Time:                        20:15:02   Log-Likelihood:                 1523.7
No. Observations:                1519   AIC:                            -3037.
Df Residuals:                    1514   BIC:                            -3011.
Df Model:                           4                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9523      0.054     17.541      0.000       0.846       1.059
v              0.0179      0.016      1.134      0.257      -0.013       0.049
ltotexpend    -0.1600      0.013    -12.354      0.000      -0.185      -0.135
age            0.0019      0.000      6.111      0.000       0.001       0.003
kids           0.0354      0.005      7.530      0.000       0.026       0.045
==============================================================================
Omnibus:                       15.678   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.471
Skew:                           0.134   Prob(JB):                     3.59e-05
Kurtosis:                       3.502   Cond. No.                         921.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [42]:
X = sm.add_constant(expendshares[["ltotexpend", "age", "kids"]])
model = sm.OLS(expendshares.salcohol, X, missing="drop").fit(cov_type="HC3")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               salcohol   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     26.64
Date:                Sun, 22 May 2022   Prob (F-statistic):           8.57e-17
Time:                        20:15:02   Log-Likelihood:                 2079.7
No. Observations:                1519   AIC:                            -4151.
Df Residuals:                    1515   BIC:                            -4130.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0091      0.019      0.478      0.633      -0.028       0.046
ltotexpend     0.0276      0.004      6.642      0.000       0.019       0.036
age           -0.0014      0.000     -6.594      0.000      -0.002      -0.001
kids          -0.0133      0.003     -3.976      0.000      -0.020      -0.007
==============================================================================
Omnibus:                      471.586   Durbin-Watson:                   2.055
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1372.772
Skew:                           1.590   Prob(JB):                    8.06e-299
Kurtosis:                       6.402   Cond. No.                         453.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [43]:
X = sm.add_constant(expendshares[["age", "kids"]])
IV2SLS(expendshares.salcohol, X, expendshares.ltotexpend, expendshares.lincome
      ).fit(cov_type="robust")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               salcohol   R-squared:                      0.0547
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0529
No. Observations:                1519   F-statistic:                    53.092
Date:                Sun, May 22 2022   P-value (F-stat)                0.0000
Time:                        20:15:02   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const      -1.983e-05     0.0403    -0.0005     0.9996     -0.0789      0.0789
age           -0.0015     0.0002    -6.4359     0.0000     -0.0019     -0.0010
kids          -0.0135     0.0033    -4.0281     0.0001     -0.0200     -0.0069
ltotexpend     0.0299     0.0095     3.1420     0.0017      0.0113      0.0486
==============================================================================

Endogenous: ltotexpend
Instruments: lincome
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7f3aade82fa0

C11.i sfood has a range of 0.057 to 0.789. It is not surprising that the share of spending on food does not contain any 0s as it would imply observations that are starving, stealing, or subsistence farming.

C11.ii Total expenditure is negatively associated with the share of food expenditure. This is intuitive since there is only so much food someone needs and only so much more that can be spent on quality. In essence, the wealthier someone becomes, the less they'll spend on food as a fraction of their total expenditure.

C11.iii lincome is statistically significant and positively correlated with ltotexpenditure. Assuming lincome is exogenous it is a valid IV.

C11.iv The IV estimate is slightly larger (-0.146 to -0.160) and still statistically significant. The IV confidence interval is larger (OLS: -0.158, -0.134; IV: -0.185, -0.135)

C11.v Results above. The p-value for the residuals is 0.224 (0.257 when using HC3 errors as suggested in Chapter 15) and so we do not find evidence that ltotexpend is endogenous. We cannot test overidentifying restrictions.

C11.vi ltotalexpenditure is positively associated with salcohol. This also makes intuitive sense as alcohol can be thought of as something of a luxury good that can be enjoyed with additional income.